In [1]:
!nvidia-smi

Sat Nov  9 16:27:30 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.127.05             Driver Version: 550.127.05     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      On  |   00000000:31:00.0 Off |                    0 |
| N/A   40C    P0             27W /   72W |    3185MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!lscpu

Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              8
On-line CPU(s) list: 0-7
Thread(s) per core:  2
Core(s) per socket:  4
Socket(s):           1
NUMA node(s):        1
Vendor ID:           AuthenticAMD
CPU family:          25
Model:               1
Model name:          AMD EPYC 7R13 Processor
Stepping:            1
CPU MHz:             1721.208
BogoMIPS:            5300.00
Hypervisor vendor:   KVM
Virtualization type: full
L1d cache:           32K
L1i cache:           32K
L2 cache:            512K
L3 cache:            16384K
NUMA node0 CPU(s):   0-7
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ht syscall nx mmxext fxsr_opt pdpe1gb rdtscp lm constant_tsc rep_good nopl nonstop_tsc cpuid extd_apicid aperfmperf tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm cmp_legacy

In [3]:
!pip install -q gensim nltk POT

In [4]:
import numpy as np
import pandas as pd
from sklearn import metrics
import torch

from sklearn.svm import SVC, LinearSVC
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from gensim.test.utils import common_texts
from gensim.models import Word2Vec

In [5]:
# # Setting up the device for GPU usage

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
device

'cuda'

In [3]:
# download the Jigsaw Toxic dataset from https://www.kaggle.com/competitions/jigsaw-toxic-comment-classification-challenge/data

In [6]:
!pwd

/home/ec2-user/SageMaker/BERT


In [7]:
!ls

BOW_and_SVM.ipynb				   sample_submission.csv.zip
embeddings_and_SVM.ipynb			   test.csv
fine_tuning_bert.ipynb				   test.csv.zip
fine-tunned-bert-model-dict.pth			   test_labels.csv
fine-tunned-bert-model.pth			   test_labels.csv.zip
jigsaw-toxic-comment-classification-challenge.zip  train.csv
sample_submission.csv				   train.csv.zip


In [7]:
!unzip -q -o jigsaw-toxic-comment-classification-challenge.zip
!unzip -q -o train.csv.zip
!unzip -q -o test.csv.zip
!unzip -q -o sample_submission.csv.zip
!unzip -q -o test_labels.csv.zip

In [8]:
df = pd.read_csv("./train.csv")
df[df['identity_hate'] == 1].head(3)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
42,001810bf8c45bf5f,You are gay or antisemmitian? \n\nArchangel WH...,1,0,1,0,1,1
105,00472b8e2d38d1ea,A pair of jew-hating weiner nazi schmucks.,1,0,1,0,1,1
176,006b94add72ed61c,I think that your a Fagget get a oife and burn...,1,0,1,1,1,1


In [7]:
df[(df['identity_hate'] == 1) & (df['severe_toxic'] == 1)].head(10)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
887,026bd33490542b2e,"you gay motherfucker i know where you live,i a...",1,1,1,0,1,1
1017,02c6e41e4b317ac3,WOULDN'T BE THE FIRST TIME BITCH. FUCK YOU I'L...,1,1,1,1,1,1
1116,03088aacc4600323,"Fuck you faggot, I don't care if i get blocked...",1,1,1,0,1,1
1312,039296aa294ee10b,"SHUT UP, YOU FAT POOP, OR I WILL KICK YOUR ASS!!!",1,1,1,1,1,1
2359,065675c12e22e3e1,You fascit Nazi! You socialist bastard! You fi...,1,1,1,0,1,1
2374,0661f992b32b3351,FUCK OF NAZI PROPAGANDA\nodpierdalcie sie jeba...,1,1,1,0,0,1
2937,07f1e0a1c10fcf28,Dumb Niggaz \n\nYall Niggas are dumb.Sean King...,1,1,1,0,1,1
3008,081a5f7b052138cf,Hey Roy. Go fukk yourself you gay bich.\n\nYa...,1,1,1,0,1,1
3469,0959d4fdaf74a050,THIS DUDE THAT BLOCKED ME IS A FUCKING COCK SU...,1,1,1,0,1,1
6063,10304d5db5fb5821,n i g g e r f a g g o t,1,1,0,0,0,1


In [9]:
new_df = df[(df['obscene'] == 0) & (df['severe_toxic'] == 0)].copy()
new_df = new_df[['comment_text', 'toxic']]
new_df

,comment_text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
...,...,...
159566,""":::::And for the second time of asking, when ...",0
159567,You should be ashamed of yourself \n\nThat is ...,0
159568,"Spitzer \n\nUmm, theres no actual article for ...",0
159569,And it looks like it was actually you who put ...,0


In [91]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt_tab')

tokenized_sents = [word_tokenize(i) for i in new_df['comment_text'].values]
word_to_vec_model = Word2Vec(sentences=tokenized_sents, vector_size=32, window=5, min_count=1, workers=4)

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [80]:
word_to_vec_model.wv.most_similar('woman', topn=20)

[('child', 0.9353910088539124),
 ('victim', 0.9194989204406738),
 ('girl', 0.9105194211006165),
 ('man', 0.9086109399795532),
 ('lawyer', 0.895413339138031),
 ('dog', 0.8859357237815857),
 ('murder', 0.8783286213874817),
 ('kid', 0.8778499364852905),
 ('mother', 0.8774956464767456),
 ('king', 0.877143144607544),
 ('crime', 0.8731101155281067),
 ('Steelpillow|Steelpillow', 0.8705015182495117),
 ('father', 0.8688550591468811),
 ('Verdy', 0.8637364506721497),
 ('Smallville', 0.8631851077079773),
 ('enemy', 0.8603680729866028),
 ('death', 0.8600096106529236),
 ('star', 0.8556082844734192),
 ('doctor', 0.8537963628768921),
 ('alive', 0.8493081331253052)]

In [15]:
word_to_vec_model.wv.most_similar(positive=['woman', 'king'], negative=['man'], topn=10)

[('president', 0.8753160238265991),
 ('youngest', 0.8732370734214783),
 ('pope', 0.8712455034255981),
 ('champion', 0.8624453544616699),
 ('camp', 0.8547214269638062),
 ('chief', 0.8500133752822876),
 ('mayor', 0.8494622707366943),
 ('elected', 0.8472843766212463),
 ('sons', 0.8467467427253723),
 ('launched', 0.8456817865371704)]

In [16]:
word_to_vec_model.wv.similar_by_word('dog')

[('adult', 0.8380416631698608),
 ('woman', 0.8333724141120911),
 ('horse', 0.8332191109657288),
 ('child', 0.8171677589416504),
 ('hot', 0.8130401968955994),
 ('fruit', 0.8110218048095703),
 ('girl', 0.8079290986061096),
 ('lady', 0.8059806823730469),
 ('fat', 0.8057582974433899),
 ('dream', 0.801310122013092)]

In [17]:
word_to_vec_model.wv.wmdistance('I have got a car'.lower().split(), 'I owe a automobile'.lower().split()), \
word_to_vec_model.wv.wmdistance('I ate a sandwich yesterday'.lower().split(), 'it will rain tomorrow'.lower().split())

(0.6886378980580458, 1.0131833271832986)

In [18]:
word_to_vec_model.wv.distance('man', 'woman'), word_to_vec_model.wv.distance('voice', 'sound')

(0.08459591865539551, 0.4198455810546875)

In [19]:
# Compute cosine similarity between two sets of keys.
word_to_vec_model.wv.n_similarity(['man'], ['woman']), word_to_vec_model.wv.n_similarity(['voice'], ['sound'])

(0.9154041, 0.5801544)

In [20]:
word_to_vec_model.wv['man']

array([-1.0930569 ,  0.25744972,  0.564636  , -0.63300264, -0.99065435,
        2.9444437 ,  0.8462044 ,  3.4011872 , -0.31788942,  1.6728779 ,
        2.9486132 ,  0.24087323,  4.50146   , -1.7103932 , -3.7561567 ,
        3.3904717 , -1.2986655 , -4.2936444 , -0.9078605 ,  3.750578  ,
        4.756841  , -1.0389476 , -0.26794696,  0.0134482 , -1.4218277 ,
       -1.6139135 ,  1.3842952 ,  2.1360643 ,  0.8305084 ,  0.55022705,
       -1.4031281 , -1.0407635 ], dtype=float32)

In [74]:
a = np.array([0, 1, 2, 3, 4, 5, 6, 7], dtype=np.float32)
print(a)
a.resize(3)
print(a)
a.resize(10)
print(a)

[0. 1. 2. 3. 4. 5. 6. 7.]
[0. 1. 2.]
[0. 1. 2. 0. 0. 0. 0. 0. 0. 0.]


In [95]:
smaller_df = new_df#[:100000]
smaller_df[smaller_df['toxic'] == 1]

vectors_list = []
X = []
max_len = 50 # defining a max len. Max Len here is the max number of tokens per sentence
for entry in smaller_df['comment_text']:
    vector = word_to_vec_model.wv[word_tokenize(entry)]
    # max_len = max(max_len, len(vector))
    vectors_list.append(vector)

for vector in vectors_list:
    resized_vector = vector.copy()
    resized_vector.resize(max_len, vector.shape[1])
    # print(vector.shape, resized_vector.shape)
    X.append(resized_vector)

X = np.array(X)
# Need to reshape to 2 dimensions otherwise SVM will fail with `ValueError: Found array with dim 3. SVC expected <= 2.`
X = X.reshape(X.shape[0], -1)

print('X.shape', X.shape)
X_train, X_test, y_train, y_test = train_test_split(X, np.array(smaller_df['toxic']), test_size=0.3, random_state=42)
pd.DataFrame(y_train).value_counts()

X.shape (151044, 1600)


0    100581
1      5149
dtype: int64

In [82]:
def is_toxic(model, text, print_score=False):
    vector = word_to_vec_model.wv[word_tokenize(text)]
    resized_vector = vector.copy()
    resized_vector.resize(max_len, vector.shape[1])
    encoded_input = np.array([resized_vector])
    encoded_input = encoded_input.reshape(encoded_input.shape[0], -1)
    # print('resized_vector', encoded_input.shape, encoded_input)
    if print_score:
        return model.predict(encoded_input)[0] == True, model.predict(encoded_input), model.decision_function(encoded_input)

    return model.predict(encoded_input)[0] == True, model.predict(encoded_input)


def prefixed_is_toxic_check(model):
    def print_is_toxic(text):
        result = is_toxic(model, text, True)
        return f'[Score: {result[2][0]:0.3f}] [Toxic: {result[0]}] {text}'

    return print_is_toxic("You are an idiot"), \
        print_is_toxic("You are a devil"), \
        print_is_toxic("You stink"), \
        print_is_toxic("What are you doing here? You are not welcome in this country"), \
        print_is_toxic("hello my friend! I love you, you know that, right?"), \
        print_is_toxic("I watched the soccer game yesterday and it was an aweful experience"), \
        print_is_toxic("Hello Dad; how are you doing?"), \
        print_is_toxic("Hi Mom!")


In [96]:
# the distribution of the data is 1 toxic for each ~20 non toxic so we need to assign a higher weight to the toxic (class 1)
linear_svm = LinearSVC(class_weight={0: 1, 1: 20})
linear_svm.fit(X_train, y_train)

y_pred = linear_svm.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      0.79      0.88     43173
           1       0.15      0.76      0.25      2141

    accuracy                           0.79     45314
   macro avg       0.57      0.78      0.56     45314
weighted avg       0.95      0.79      0.85     45314



In [97]:
prefixed_is_toxic_check(linear_svm)

('[Score: 0.835] [Toxic: True] You are an idiot',
 '[Score: 0.866] [Toxic: True] You are a devil',
 '[Score: 0.578] [Toxic: True] You stink',
 '[Score: 0.323] [Toxic: True] What are you doing here? You are not welcome in this country',
 '[Score: 0.838] [Toxic: True] hello my friend! I love you, you know that, right?',
 '[Score: 0.322] [Toxic: True] I watched the soccer game yesterday and it was an aweful experience',
 '[Score: -0.033] [Toxic: False] Hello Dad; how are you doing?',
 '[Score: 0.037] [Toxic: True] Hi Mom!')

In [ ]:
# the distribution of the data is 1 toxic for each ~20 non toxic so we need to assign a higher weight to the toxic (class 1)
svm = SVC(class_weight={0: 1, 1: 20})
svm.fit(X_train, y_train)

y_pred = svm.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
prefixed_is_toxic_check(svm)